In [1]:

import torch
from yolov5.detect import run
from utils_main.utils import ResizeImages
from siamese_triplet.datasets import LeopardDataset
from siamese_triplet.networks import EmbeddingNet
from siamese_triplet.networks import EmbeddingWithSoftmaxNet
from siamese_triplet.networks import MultiPartEmbeddingNet
from siamese_triplet.networks import MultiPartEmbeddingWithSoftmaxNet
from siamese_triplet.networks import extract_embeddings
from siamese_triplet.networks import InferenceNetwork

In [2]:
#Yolov5 folders
CROP_FOLDER = '../dataset/'
YOLOV5_FOLDER = './yolov5/'
YOLOV5_WEIGHTS_FOLDER = YOLOV5_FOLDER + 'yolov5_weights/best.pt'
INPUT_IMAGE_FOLDER = '/home/user1/work/W210/datasets/temp/'

# Siamese network files/folder
model_file_name = 'leop_id_model_july_28.pt'
centroid_file_name = 'centroid_july_28.pt'
dict_file_name = 'leopard_label_dict.pt'
model_path = f"./siamese_triplet/weights/{model_file_name}"    
centroid_path = f"./siamese_triplet/weights/{centroid_file_name}"   
labels_dict_path = f"./siamese_triplet/weights/{dict_file_name}" 

# input path to identification
RESIZE_IMAGE_PATH = CROP_FOLDER+'resize/'
   

In [ ]:
# Run Yolov5 to generate cropped images
run(weights=YOLOV5_WEIGHTS_FOLDER,
    conf_thres= 0.75,
    nosave=True,
    data = YOLOV5_FOLDER+'/data/custom.yaml',
    project = CROP_FOLDER,
    name = '',
    source = INPUT_IMAGE_FOLDER,
    save_crop=True)

In [3]:
# Generate resize images from cropped images
resize_images = ResizeImages(in_path=CROP_FOLDER+'crops/',out_path=CROP_FOLDER+'resize/leop_xx')
#resize_images.run()

In [4]:
# Initialize Siamese network model and load weights
MULTI_EMBEDDING = True
softmax = True
cuda = torch.cuda.is_available()

model = EmbeddingWithSoftmaxNet(num_classes=64)
if cuda:
    model.cuda()

model.load_state_dict(torch.load(model_path))
model.centroids = torch.load(centroid_path)
labels_dict = torch.load(labels_dict_path)


In [5]:
#Run identification Network

infer_network = InferenceNetwork(infer_path=RESIZE_IMAGE_PATH,
                                 model = model,
                                 labels_dict = labels_dict,
                                 infer_labels = resize_images.labels,
                                 ood_reject_thresh = 0.675,
                                 cuda = cuda
                                 )
pred_ind, pred_label = infer_network.run()
print(pred_label)

['leop_0', 'leop_0', 'leop_0', 'leop_0', 'leop_0', 'leop_0', 'leop_0', 'leop_UKN']
